In [ ]:
# Definimos la arquitectura a utilizar por el algoritmo 

In [24]:
IMAGE_SIZE=224
ARCHITECTURE="mobilenet_0.50_%s" %(IMAGE_SIZE) 
print('ARCHITECTURE:', ARCHITECTURE)

ARCHITECTURE: mobilenet_0.50_224


In [ ]:
# Ejecutamos el algoritmo de entrenamiento de tensorflow.
# Parámetros importantes
# architecture: Arquitectura a utilizar para el entrenamiento.
# image_dir: carpeta donde se encuentran las imágenes a clasificar

In [26]:
%run -i "retrain.py" \
   --bottleneck_dir=tf_files/bottlenecks --how_many_training_steps=500 \
   --model_dir=tf_files/models/  --summaries_dir=tf_files/training_summaries/ \
   --output_graph=tf_files/retrained_graph.pb   --output_labels=tf_files/retrained_labels.txt --architecture="{ARCHITECTURE}"  \
   --image_dir=data

INFO:tensorflow:Looking for images in 'maradona'
INFO:tensorflow:Looking for images in 'messi'
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.

INFO:tensorflow:2018-06-07 09:12:16.025139: Step 0: Train accuracy = 78.0%
INFO:tensorflow:2018-06-07 09:12:16.027121: Step 0: Cross entropy = 0.391156
INFO:tensorflow:2018-06-07 09:12:16.223367: Step 0: Validation accuracy = 90.0% (N=100)
INFO:tensorflow:2018-06-07 09:12:16.618984: Step 10: Train accuracy = 71.0%
INFO:tensorflow:2018-06-07 09:12:16.620076: Step 10: Cross entropy = 2.843077
INFO:tensorflow:2018-06-07 09:12:16.666334: Step 10: Validation accuracy = 64.0% (N=100)
INFO:tensorflow:2018-06-07 09:12:17.061183: Step 20: Train accuracy = 81.0%
INFO:tensorflow:2018-06-07 09:12:17.061815: Step 20: Cross entropy = 1.194225
INFO:tensorflow:2018-06-07 09:12:17.108368: Step 20: Validation accuracy = 7

INFO:tensorflow:2018-06-07 09:12:30.373120: Step 310: Cross entropy = 0.000565
INFO:tensorflow:2018-06-07 09:12:30.419340: Step 310: Validation accuracy = 86.0% (N=100)
INFO:tensorflow:2018-06-07 09:12:30.869710: Step 320: Train accuracy = 100.0%
INFO:tensorflow:2018-06-07 09:12:30.870388: Step 320: Cross entropy = 0.000570
INFO:tensorflow:2018-06-07 09:12:30.913655: Step 320: Validation accuracy = 86.0% (N=100)
INFO:tensorflow:2018-06-07 09:12:31.333725: Step 330: Train accuracy = 100.0%
INFO:tensorflow:2018-06-07 09:12:31.334457: Step 330: Cross entropy = 0.000585
INFO:tensorflow:2018-06-07 09:12:31.381686: Step 330: Validation accuracy = 89.0% (N=100)
INFO:tensorflow:2018-06-07 09:12:31.789057: Step 340: Train accuracy = 100.0%
INFO:tensorflow:2018-06-07 09:12:31.790102: Step 340: Cross entropy = 0.000817
INFO:tensorflow:2018-06-07 09:12:31.831525: Step 340: Validation accuracy = 92.0% (N=100)
INFO:tensorflow:2018-06-07 09:12:32.229218: Step 350: Train accuracy = 100.0%
INFO:tensorf

In [ ]:
# Testeamos el entrenamiento. En este caso esperamos que infiera que el personaje de la imagen sea Maradona.

In [30]:
%run -i label_image.py --graph=tf_files/retrained_graph.pb --labels=tf_files/retrained_labels.txt --image=data/maradona.jpg


Evaluation time (1-image): 0.192s

maradona (score=0.99995)
messi (score=0.00005)


In [31]:
# Luego del entrenamiento, convertiremos el grafo generado a un modelo CoreML que puede correr en dispositivos con IOS.

In [32]:
import tfcoreml as tf_converter


In [33]:
model_name = 'maradona.mlmodel'

tf_converter.convert(tf_model_path = 'tf_files/retrained_graph.pb',
                     mlmodel_path = model_name,
                     image_input_names = 'input:0',
                     class_labels = 'tf_files/retrained_labels.txt',
                     output_feature_names = ['final_result:0'],
                     red_bias = -1,
                     green_bias = -1,
                     blue_bias = -1,
                     image_scale = 2.0/255.0)

Shapes not found for 4 tensors. Executing graph to determine shapes. 
Automatic shape interpretation succeeded for input blob input:0
1/560: Analysing op name: MobilenetV1/Predictions/Reshape/shape ( type:  Const )
2/560: Analysing op name: MobilenetV1/Logits/Conv2d_1c_1x1/biases ( type:  Const )
3/560: Analysing op name: MobilenetV1/Logits/Conv2d_1c_1x1/biases/read ( type:  Identity )
4/560: Analysing op name: MobilenetV1/Logits/Conv2d_1c_1x1/weights ( type:  Const )
5/560: Analysing op name: MobilenetV1/Logits/Conv2d_1c_1x1/weights/read ( type:  Identity )
6/560: Analysing op name: MobilenetV1/MobilenetV1/Conv2d_13_pointwise/BatchNorm/batchnorm/add/y ( type:  Const )
7/560: Analysing op name: MobilenetV1/Conv2d_13_pointwise/BatchNorm/moving_variance ( type:  Const )
8/560: Analysing op name: MobilenetV1/Conv2d_13_pointwise/BatchNorm/moving_variance/read ( type:  Identity )
9/560: Analysing op name: MobilenetV1/MobilenetV1/Conv2d_13_pointwise/BatchNorm/batchnorm/add ( type:  Add )
10/

496/560: Analysing op name: MobilenetV1/MobilenetV1/Conv2d_7_pointwise/BatchNorm/batchnorm/mul_1 ( type:  Mul )
497/560: Analysing op name: MobilenetV1/MobilenetV1/Conv2d_7_pointwise/BatchNorm/batchnorm/add_1 ( type:  Add )
498/560: Analysing op name: MobilenetV1/MobilenetV1/Conv2d_7_pointwise/Relu6 ( type:  Relu6 )
499/560: Analysing op name: MobilenetV1/MobilenetV1/Conv2d_8_depthwise/depthwise ( type:  DepthwiseConv2dNative )
500/560: Analysing op name: MobilenetV1/MobilenetV1/Conv2d_8_depthwise/BatchNorm/batchnorm/mul_1 ( type:  Mul )
501/560: Analysing op name: MobilenetV1/MobilenetV1/Conv2d_8_depthwise/BatchNorm/batchnorm/add_1 ( type:  Add )
502/560: Analysing op name: MobilenetV1/MobilenetV1/Conv2d_8_depthwise/Relu6 ( type:  Relu6 )
503/560: Analysing op name: MobilenetV1/MobilenetV1/Conv2d_8_pointwise/convolution ( type:  Conv2D )
504/560: Analysing op name: MobilenetV1/MobilenetV1/Conv2d_8_pointwise/BatchNorm/batchnorm/mul_1 ( type:  Mul )
505/560: Analysing op name: Mobilenet

input {
  name: "input__0"
  type {
    imageType {
      width: 224
      height: 224
      colorSpace: RGB
    }
  }
}
output {
  name: "final_result__0"
  type {
    dictionaryType {
      stringKeyType {
      }
    }
  }
}
output {
  name: "classLabel"
  type {
    stringType {
    }
  }
}
predictedFeatureName: "classLabel"
predictedProbabilitiesName: "final_result__0"

In [ ]:
# Done!